In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class BasicBlock(nn.Module):
  # block 안에서 plane 수 증가 비율
  expansion = 1

  # in_planes : input, planes: output
  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,stride=1,padding=1,bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    # identity mapping
    self.shortcut = nn.Sequential()

    # shortcut connection 시 shape가 맞지 않을 때(conv_n 이 conv_n+1로 넘어갈 때)
    # projection mapping
    if stride != 1 or in_planes != self.expansion * planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion * planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))

    # shortcut connection
    out += self.shortcut(x)

    out = F.relu(out)
    return out


In [3]:
class Bottleneck(nn.Module):
  # Bottleneck에서는 plane이 마지막에 4배로 팽창
  expansion = 4

  def __init__(self,in_planes, planes, stride=1):
    super(Bottleneck,self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes,kernel_size=1,bias=False)
    self.bn1 = nn.BatchNorm2d(planes)

    # 사이즈 유지 위해 패딩 1
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
    self.bn3 = nn.BatchNorm2d(self.expansion * planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion * planes:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = F.relu(self.bn2(self.conv2(out)))
    out = self.bn3(self.conv3(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out

In [18]:
class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    # 64부터 시작
    self.in_planes = 64
    self.dropout = nn.Dropout()
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)

    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    # 마지막 클래스 예측
    self.linear = nn.Linear(512 * block.expansion, num_classes)
  
  def _make_layer(self, block, planes, num_blocks, stride):
    # 각 size 처음 stride만 2, 이후 모두 1
    strides = [stride] + [1] * (num_blocks-1)
    layers = []

    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.dropout(out)
    out = self.layer1(out)
    out = self.dropout(out)
    out = self.layer2(out)
    out = self.dropout(out)
    out = self.layer3(out)
    out = self.dropout(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out,4)
    out = out.view(out.size(0),-1)
    out = self.linear(out)
    return out

In [19]:
def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

def ResNet34():
  return ResNet(BasicBlock, [3, 4, 6, 3])

def ResNet50():
  return ResNet(Bottleneck, [3, 4, 6, 3])

def ResNet101():
  return ResNet(Bottleneck, [3, 4, 23, 3])

def ResNet152():
  return ResNet(Bottleneck, [3, 8, 36, 3])